In [1]:
# ----------------------------------------------------------------------
# Numenta Platform for Intelligent Computing (NuPIC)
# Copyright (C) 2019, Numenta, Inc.  Unless you have an agreement
# with Numenta, Inc., for a separate license for this software code, the
# following terms and conditions apply:
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Affero Public License version 3 as
# published by the Free Software Foundation.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
# See the GNU Affero Public License for more details.
#
# You should have received a copy of the GNU Affero Public License
# along with this program.  If not, see http://www.gnu.org/licenses.
#
# http://numenta.org/licenses/
# ----------------------------------------------------------------------

In [2]:
!pip install git+https://github.com/numenta/nupic.torch.git#egg=nupic.torch
!pip install torch torchvision

In [3]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm_notebook as tqdm

SEED = 18
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
def train(model, loader, optimizer, criterion):
    """
    Train the model using given dataset loader. 
    Called on every epoch.
    :param model: pytorch model to be trained
    :type model: torch.nn.Module
    :param loader: dataloader configured for the epoch.
    :type loader: :class:`torch.utils.data.DataLoader`
    :param optimizer: Optimizer object used to train the model.
    :type optimizer: :class:`torch.optim.Optimizer`
    :param criterion: loss function to use
    :type criterion: function
    """
    model.train()
    for batch_idx, (data, target) in enumerate(tqdm(loader, leave=False)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()


def test(model, loader, criterion):
    """
    Evaluate pre-trained model using given dataset loader.
    Called on every epoch.
    :param model: Pretrained pytorch model
    :type model: torch.nn.Module
    :param loader: dataloader configured for the epoch.
    :type loader: :class:`torch.utils.data.DataLoader`
    :param criterion: loss function to use
    :type criterion: function
    :return: Dict with "accuracy", "loss" and "total_correct"
    """
    model.eval()
    loss = 0
    total_correct = 0
    with torch.no_grad():
        for data, target in tqdm(loader, leave=False):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss += criterion(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            total_correct += pred.eq(target.view_as(pred)).sum().item()
    
    return {"accuracy": total_correct / len(loader.dataset), 
            "loss": loss / len(loader.dataset), 
            "total_correct": total_correct}

### Parameters

In [5]:
# Training parameters
LEARNING_RATE = 0.02
LEARNING_RATE_GAMMA = 0.8
MOMENTUM = 0.0
EPOCHS = 15
FIRST_EPOCH_BATCH_SIZE = 4
TRAIN_BATCH_SIZE = 64
TEST_BATCH_SIZE = 1000

### Create Sparse MNIST model

There are 2 ways to create **nupic.torch** sparse models. You can import the models from **nupic.torch.models** or use pytorch's [torch.hub](https://pytorch.org/docs/stable/hub.html) API.

In this example we will import the models. 

In [6]:
from nupic.torch.models import MNISTSparseCNN
# For this example we will use the default values. 
# See MNISTSparseCNN documentation for all possible parameters and their values.
model = MNISTSparseCNN().to(device)
print(model)

MNISTSparseCNN(
  (cnn1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (cnn1_maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn1_kwinner): KWinners2d(channels=32, n=0, percent_on=0.087, boost_strength=1.5, boost_strength_factor=0.85, k_inference_factor=1.5, duty_cycle_period=1000)
  (cnn2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (cnn2_maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2_kwinner): KWinners2d(channels=64, n=0, percent_on=0.293, boost_strength=1.5, boost_strength_factor=0.85, k_inference_factor=1.5, duty_cycle_period=1000)
  (flatten): Flatten()
  (linear): SparseWeights(
    weight_sparsity=0.3
    (module): Linear(in_features=1024, out_features=700, bias=True)
  )
  (linear_kwinner): KWinners(n=700, percent_on=0.143, boost_strength=1.5, boost_strength_factor=0.85, k_inference_factor=1.5, duty_cycle_period=1000)
  (output): Linear(in_features=700, out_features=10, bias=Tru

### Load MNIST Dataset

In [7]:
normalize = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.MNIST('data', train=True, download=True, transform=normalize)
test_dataset = datasets.MNIST('data', train=False, transform=normalize)

# Configure data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, shuffle=True)
first_loader = torch.utils.data.DataLoader(train_dataset, batch_size=FIRST_EPOCH_BATCH_SIZE, shuffle=True)

### Train
On the first epoch we use smaller batch size to calculate the duty cycles used by the k-winner function. Once the duty cycles stabilize we can use larger batch sizes.

In [8]:
sgd = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
lr_scheduler = optim.lr_scheduler.StepLR(sgd, step_size=1, gamma=LEARNING_RATE_GAMMA)
train(model=model, loader=first_loader, optimizer=sgd, criterion=F.nll_loss)
lr_scheduler.step()

After each epoch we rezero the weights to keep the initial sparsity constant during training. We also apply the boost strength factor after each epoch

In [9]:
%%capture
from nupic.torch.modules import rezero_weights, update_boost_strength
model.apply(rezero_weights)
model.apply(update_boost_strength)

Test and print results

In [10]:
test(model=model, loader=test_loader, criterion=F.nll_loss)

{'accuracy': 0.9823, 'loss': 0.05553661346435547, 'total_correct': 9823}

At this point the duty cycles should be stable and we can train on larger batch sizes

In [11]:
for epoch in range(1, EPOCHS):
    train(model=model, loader=train_loader, optimizer=sgd, criterion=F.nll_loss)
    lr_scheduler.step()
    model.apply(rezero_weights)
    model.apply(update_boost_strength)
    results = test(model=model, loader=test_loader, criterion=F.nll_loss)
    print(results)

{'accuracy': 0.99, 'loss': 0.03087631359100342, 'total_correct': 9900}


{'accuracy': 0.9913, 'loss': 0.028131033515930177, 'total_correct': 9913}


{'accuracy': 0.9912, 'loss': 0.02829796772003174, 'total_correct': 9912}


{'accuracy': 0.9901, 'loss': 0.03012564697265625, 'total_correct': 9901}


{'accuracy': 0.9909, 'loss': 0.0279187762260437, 'total_correct': 9909}


...

{'accuracy': 0.9915, 'loss': 0.02673978271484375, 'total_correct': 9915}


{'accuracy': 0.9916, 'loss': 0.026827363300323485, 'total_correct': 9916}


{'accuracy': 0.9914, 'loss': 0.02654646396636963, 'total_correct': 9914}


{'accuracy': 0.9916, 'loss': 0.026357748794555665, 'total_correct': 9916}


{'accuracy': 0.9917, 'loss': 0.026316686820983887, 'total_correct': 9917}


### Noise
Add noise to the input and check the test accuracy

In [12]:
class RandomNoise(object):
    """
    An image transform that adds noise to random pixels in the image.
    """
    def __init__(self, noise_level=0.0, white_value=0.1307 + 2*0.3081):
        """
        :param noise_level:
          From 0 to 1. For each pixel, set its value to white_value with this
          probability. Suggested white_value is 'mean + 2*stdev'
        """
        self.noise_level = noise_level
        self.white_value = white_value

    def __call__(self, image):
        a = image.view(-1)
        num_noise_bits = int(a.shape[0] * self.noise_level)
        noise = np.random.permutation(a.shape[0])[0:num_noise_bits]
        a[noise] = self.white_value
        return image

In [13]:
noise_score = 0
for noise in [0.0, 0.05, 0.10, 0.15, 0.20, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]:
    noise_transform = transforms.Compose([transforms.ToTensor(), RandomNoise(noise), 
                                      transforms.Normalize((0.1307,), (0.3081,))])
    noise_dataset = datasets.MNIST('data', train=False, transform=noise_transform)
    noise_loader = torch.utils.data.DataLoader(noise_dataset, 
                                               batch_size=TEST_BATCH_SIZE, 
                                               shuffle=True)

    results = test(model=model, loader=noise_loader, criterion=F.nll_loss)
    noise_score += results["total_correct"]
    print(noise, ":", results)

0.0 : {'accuracy': 0.9917, 'loss': 0.02631667709350586, 'total_correct': 9917}


0.05 : {'accuracy': 0.9899, 'loss': 0.032843858909606935, 'total_correct': 9899}


0.1 : {'accuracy': 0.9855, 'loss': 0.0439798023223877, 'total_correct': 9855}


0.15 : {'accuracy': 0.9829, 'loss': 0.05501823501586914, 'total_correct': 9829}


0.2 : {'accuracy': 0.976, 'loss': 0.07691587409973144, 'total_correct': 9760}


...

0.3 : {'accuracy': 0.9499, 'loss': 0.15266261291503908, 'total_correct': 9499}


0.35 : {'accuracy': 0.9305, 'loss': 0.21989019317626954, 'total_correct': 9305}


0.4 : {'accuracy': 0.9015, 'loss': 0.30402026062011717, 'total_correct': 9015}


0.45 : {'accuracy': 0.8543, 'loss': 0.4642761535644531, 'total_correct': 8543}


0.5 : {'accuracy': 0.7951, 'loss': 0.6449000061035156, 'total_correct': 7951}


In [14]:
print("noise_score:", noise_score)

noise_score: 103228
